In [1]:
import warnings
warnings.simplefilter('ignore')

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import datetime as dt
import time
from sklearn.metrics import auc, roc_curve, classification_report
from scipy import stats

#import h2o
#from h2o.frame import H2OFrame
#from h2o.estimators.random_forest import H2ORandomForestEstimator
#from h2o.grid.grid_search import H2OGridSearch

%matplotlib inline
sns.set(style="white",context="talk")

# 1. Load the dataset

In [2]:
data = pd.read_json('song.json')
data['time_played'] = pd.to_datetime(data['time_played'])
data['user_sign_up_date'] = pd.to_datetime(data['user_sign_up_date'])
data.head()

,id,user_id,user_state,user_sign_up_date,song_played,time_played
0,GOQMMKSQQH,122,Louisiana,2015-05-16,Hey Jude,2015-06-11 21:51:35
1,HWKKBQKNWI,3,Ohio,2015-05-01,We Can Work It Out,2015-06-06 16:49:19
2,DKQSXVNJDH,35,New Jersey,2015-05-04,Back In the U.S.S.R.,2015-06-14 02:11:29
3,HLHRIDQTUW,126,Illinois,2015-05-16,P.s. I Love You,2015-06-08 12:26:10
4,SUKJCSBCYW,6,New Jersey,2015-05-01,Sgt. Pepper's Lonely Hearts Club Band,2015-06-28 14:57:00


In [3]:
def view(df):
    # Exploring the data types, number of unique values and missing values
    cols = df.columns
    print(f"The dataset consists of {df.shape[0]} rows and {df.shape[1]} columns")
    print(f"The columns are: {df.columns.tolist()}")
    return pd.concat([pd.DataFrame({"data_types":df.dtypes, 
              "value_counts": df.nunique(),
             "null_counts": df.isnull().sum()}).T,df.iloc[0:3,:]],axis = 0)
view(data)

The dataset consists of 4000 rows and 6 columns
The columns are: ['id', 'user_id', 'user_state', 'user_sign_up_date', 'song_played', 'time_played']


,id,user_id,user_state,user_sign_up_date,song_played,time_played
data_types,object,int64,object,datetime64[ns],object,datetime64[ns]
value_counts,4000,196,41,20,97,3997
null_counts,0,0,0,0,0,0
0,GOQMMKSQQH,122,Louisiana,2015-05-16 00:00:00,Hey Jude,2015-06-11 21:51:35
1,HWKKBQKNWI,3,Ohio,2015-05-01 00:00:00,We Can Work It Out,2015-06-06 16:49:19
2,DKQSXVNJDH,35,New Jersey,2015-05-04 00:00:00,Back In the U.S.S.R.,2015-06-14 02:11:29


### Deal with date/time factors

In [4]:
def add_date_factors(df, ts_col, is_datetime):
    '''
    Assumption: the currect ts_col is a timestamp datatype
    
    Input:
    df (dataframe, target dataframe to be transformed)
    ts_col(string, name of the timestamp column)
    is_datetime(boolean, whether ts_col is datetime object or not)
    
    Output:
    This function generates hour, hour_of_day, date, weekday, week_of_month, months, year
    '''
    if is_datetime:
        pass
    else:
        df[ts_col] = list(map(lambda x: datetime.fromisoformat(x),df[ts_col].replace(":60",":59", regex=True)))

    # Hour
    df["hour"] = list(map(lambda x: x.hour, df[ts_col]))    

    # Hour of day
    def get_hourday(x):
        if 0<= x < 6:
            return "midnight"
        elif 6<= x < 12:
            return "morning"
        elif 12<= x < 18:
            return "afternoon"
        else:
            return "night"
    df["hourday"] = list(map(lambda x: get_hourday(x), df["hour"]))
        
    # Date
    df["date"] =  list(map(lambda day:day.date(), df[ts_col]))

    # Weekday
    wkday_dict = {1:"Mon",2:"Tue",3:"Wed",4:"Thu",5:"Fri",6:"Sat",7:"Sun"}
    df["weekday_index"] = list(map(lambda x: x.date().weekday()+1, df[ts_col]))
    df['weekday'] = list(map(lambda x: wkday_dict[x],df["weekday_index"]))

    # Week_of_month (assuming 4 weeks in a month)
    def get_week(day):
        if 1<= day.day < 8:
            return ("first_week",1)
        elif 8<= day.day < 15:
            return ("second_week",2)
        elif 15<= day.day < 22:
            return ("third_week",3)
        else:
            return ("last_week",4)
    df["week"] = list(map(lambda day: get_week(day)[0], df[ts_col]))
    df["week_index"] = list(map(lambda day: get_week(day)[1], df[ts_col]))

    # Month
    month_dict = {1:"Jan",2:"Feb",3:"Mar",4:"Apr",5:"May",6:"Jun",7:"Jul",8:"Aug",9:"Sep",10:"Oct",11:"Nov",12:"Dec"}
    df["month_index"] = list(map(lambda day: day.month, df[ts_col]))
    df["month"] = list(map(lambda m: month_dict[m], df["month_index"]))

    # Year
    df["year"] = list(map(lambda x:x.year,df[ts_col]))
    return df

In [5]:
data = add_date_factors(data, "time_played",True)
# All the data is in June,2015, so I dropped the year and month columns
data = data.drop(["year","month","month_index"],axis = 1)

## Q1: What are the top 3 and the bottom 3 states in terms of number of users?

In [6]:
data.groupby("user_state")["user_id"].count().sort_values(ascending = False)

user_state
New York          469
California        425
Texas             230
Ohio              209
Florida           180
Pennsylvania      179
North Carolina    154
Illinois          149
Georgia           135
Missouri          127
New Jersey        117
Maryland          112
Louisiana         105
Alabama           104
Tennessee         102
Wisconsin          95
Massachusetts      91
Mississippi        85
South Carolina     85
Michigan           80
Kentucky           78
Oregon             62
Alaska             58
Indiana            55
Colorado           54
Oklahoma           49
Minnesota          42
Washington         41
West Virginia      38
Utah               38
Nebraska           36
Arkansas           34
Rhode Island       27
North Dakota       26
Idaho              26
Iowa               23
Arizona            22
Virginia           17
New Mexico         17
Connecticut        16
Kansas              8
Name: user_id, dtype: int64

## Q2: What are the top 3 and the bottom 3 states in terms of user engagement? 
You can choose how to mathematically define user engagement. What the CEO cares about here is in which states users are using the product a lot/very little

### Engagement metrics (1): average number of song listened per user

In [7]:
state_eng = pd.DataFrame(data.groupby(by = ["user_state","user_id"])["song_played"].count().
             reset_index().groupby("user_state").song_played.mean().sort_values(ascending = False))
state_eng.head(5)

,song_played
user_state,
Nebraska,36.000000
Alaska,29.000000
Mississippi,28.333333
South Carolina,28.333333
Rhode Island,27.000000


In [8]:
state_eng.tail(3)

,song_played
user_state,
Minnesota,10.5
Virginia,8.5
Kansas,8.0


### Engagement metrics (2): average number of unique song listened per user

In [9]:
state_eng2 = pd.DataFrame(data.groupby(by = ["user_state","user_id"])["song_played"].nunique().
    reset_index().groupby("user_state").song_played.mean().sort_values(ascending = False))
state_eng2.head(5)

,song_played
user_state,
Nebraska,24.0
Rhode Island,21.0
Alabama,18.5
Kentucky,18.0
North Carolina,18.0


In [10]:
state_eng2.tail(3)

,song_played
user_state,
Minnesota,7.5
Kansas,7.0
Virginia,6.0


## Q3: Generate a list of first signed-up users
The CEO wants to send a gift to the first user who signed-up for each state. That is, the
first user who signed-up from California, from Oregon, etc. Can you give him a list of
those users?

In [11]:
grouped = data.groupby(by = ["user_state","user_id"]).user_sign_up_date.min().reset_index()

In [12]:
from pandasql import sqldf
pysqldf = lambda q: sqldf(q, globals())

In [13]:
q = """
    SELECT user_state, user_id, user_sign_up_date
    FROM (
        SELECT user_state, user_id, user_sign_up_date, ROW_NUMBER() OVER (PARTITION BY user_state ORDER BY user_sign_up_date) AS ranking
        FROM grouped) t
    WHERE ranking = 1;
    """
first_signed_up_users = pysqldf(q)

In [14]:
first_signed_up_users["user_sign_up_date"] = pd.to_datetime(first_signed_up_users["user_sign_up_date"])
first_signed_up_users

,user_state,user_id,user_sign_up_date
0,Alabama,5,2015-05-01
1,Alaska,106,2015-05-12
2,Arizona,105,2015-05-12
3,Arkansas,78,2015-05-08
4,California,39,2015-05-04
5,Colorado,166,2015-05-19
6,Connecticut,127,2015-05-16
7,Florida,41,2015-05-04
8,Georgia,16,2015-05-02
9,Idaho,165,2015-05-19


## Q4: Recommendation modeling
Build a function that takes as an input any of the songs in the data and returns the most likely song to be listened next. That is, if, for instance, a user is currently listening to "Eight Days A Week", which song has the highest probability of being played right after it
by the same user? This is going to be v1 of a song recommendation model

In [15]:
view(data)

The dataset consists of 4000 rows and 13 columns
The columns are: ['id', 'user_id', 'user_state', 'user_sign_up_date', 'song_played', 'time_played', 'hour', 'hourday', 'date', 'weekday_index', 'weekday', 'week', 'week_index']


,id,user_id,user_state,user_sign_up_date,song_played,time_played,hour,hourday,date,weekday_index,weekday,week,week_index
data_types,object,int64,object,datetime64[ns],object,datetime64[ns],int64,object,object,int64,object,object,int64
value_counts,4000,196,41,20,97,3997,24,4,28,7,7,4,4
null_counts,0,0,0,0,0,0,0,0,0,0,0,0,0
0,GOQMMKSQQH,122,Louisiana,2015-05-16 00:00:00,Hey Jude,2015-06-11 21:51:35,21,night,2015-06-11,4,Thu,second_week,2
1,HWKKBQKNWI,3,Ohio,2015-05-01 00:00:00,We Can Work It Out,2015-06-06 16:49:19,16,afternoon,2015-06-06,6,Sat,first_week,1
2,DKQSXVNJDH,35,New Jersey,2015-05-04 00:00:00,Back In the U.S.S.R.,2015-06-14 02:11:29,2,midnight,2015-06-14,7,Sun,second_week,2


### collaborative filtering algorithm

There should be a lot of different methods for this question. The simplest method is to choose the most popular songs. In addition, **k-nearest neighbor (KNN) method** can also be used. More advancely, collaborative filtering method can be implemented.

Here, I implement a simple version of **collaborative filtering algorithm** for song recommendation. More specifically, the similarity of two songs is calculate using the number of users whole listen them together.

In [16]:
# Step 1: build the Song-User matrix
song_user = data.groupby(['song_played', 'user_id'])['id'].count().unstack(fill_value=0)
# doesn't care about the number of plays, only care about whether this user has listened to it 
# because 0 and 1 can be normalized to probability, just like the logistics case
song_user = (song_user > 0).astype(int)
song_user.head()

user_id,1,2,3,4,5,6,7,8,9,10,...,191,192,193,194,195,196,197,198,199,200
song_played,,,,,,,,,,,,,,,,,,,,,
A Day In The Life,0,0,1,1,0,1,0,0,0,0,...,0,0,1,1,0,1,0,0,1,0
A Hard Day's Night,0,0,0,0,0,1,0,0,1,0,...,0,0,0,0,1,0,0,0,0,0
A Saturday Club Xmas/Crimble Medley,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ANYTIME AT ALL,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Across The Universe,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
song_user.T.head()

song_played,A Day In The Life,A Hard Day's Night,A Saturday Club Xmas/Crimble Medley,ANYTIME AT ALL,Across The Universe,All My Loving,All You Need Is Love,And Your Bird Can Sing,BAD BOY,Baby You're A Rich Man,...,We Can Work It Out,When I'm 64,While My Guitar Gently Weeps,Wild Honey Pie,With a Little Help From My Friends,YOUR MOTHER SHOULD KNOW,Yellow Submarine,Yesterday,You Never Give Me Your Money,You're Going To Lose That Girl
user_id,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,1,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,1,1,0
3,1,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
4,1,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,1,0,0,0,0,0


In [18]:
# Step 2: build song-song similarity matrix
from sklearn.preprocessing import normalize
song_user_norm = normalize(song_user, axis=1)  # normalize the song-user matrix

# calculate the similarity matrix (np.dot is just common matrix multiplication)
# The result is a 97*97 matrix (number of songs is 97 in this dataset)
similarity = np.dot(song_user_norm, song_user_norm.T)  
# Each cell is the percentage of people among 196 users who have listened to both of the two songs
#similarity = np.dot(song_user, song_user.T)  

similarity_df = pd.DataFrame(similarity, index=song_user.index, columns=song_user.index)
similarity_df.head()

song_played,A Day In The Life,A Hard Day's Night,A Saturday Club Xmas/Crimble Medley,ANYTIME AT ALL,Across The Universe,All My Loving,All You Need Is Love,And Your Bird Can Sing,BAD BOY,Baby You're A Rich Man,...,We Can Work It Out,When I'm 64,While My Guitar Gently Weeps,Wild Honey Pie,With a Little Help From My Friends,YOUR MOTHER SHOULD KNOW,Yellow Submarine,Yesterday,You Never Give Me Your Money,You're Going To Lose That Girl
song_played,,,,,,,,,,,,,,,,,,,,,
A Day In The Life,1.000000,0.263117,0.138675,0.14825,0.131559,0.299572,0.294174,0.098058,0.228802,0.200160,...,0.525213,0.113228,0.585429,0.278503,0.539411,0.087706,0.328897,0.377141,0.163430,0.0
A Hard Day's Night,0.263117,1.000000,0.000000,0.00000,0.100000,0.146385,0.111803,0.000000,0.000000,0.091287,...,0.305788,0.129099,0.266996,0.000000,0.157027,0.000000,0.050000,0.215003,0.074536,0.0
A Saturday Club Xmas/Crimble Medley,0.138675,0.000000,1.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.161165,0.000000,0.000000,0.182574,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
ANYTIME AT ALL,0.148250,0.000000,0.000000,1.00000,0.000000,0.164957,0.094491,0.125988,0.000000,0.000000,...,0.172292,0.000000,0.188044,0.097590,0.176950,0.000000,0.000000,0.103835,0.000000,0.0
Across The Universe,0.131559,0.100000,0.000000,0.00000,1.000000,0.097590,0.000000,0.000000,0.000000,0.000000,...,0.101929,0.000000,0.133498,0.000000,0.104685,0.000000,0.000000,0.061430,0.000000,0.0


In [19]:
# Step 3: find the top-k most similar songs
def find_topk(song, similarity, k=1):
    df = similarity.loc[song].sort_values(ascending=False)[1:k + 1].reset_index()
    df = df.rename(columns={'song_played': 'Song', song: 'Similarity'})
    
    return df

In [20]:
# Example: find the top 10 similar song for 'A Day In The Life'
df = find_topk(song='A Day In The Life', similarity=similarity_df, k=10)
df

,Song,Similarity
0,Revolution,0.710186
1,Come Together,0.696557
2,Get Back,0.676128
3,Hello Goodbye,0.617065
4,Back In the U.S.S.R.,0.614700
5,Let It Be,0.601105
6,Hey Jude,0.599145
7,While My Guitar Gently Weeps,0.585429
8,Lucy In The Sky With Diamonds,0.577453
9,Here Comes The Sun,0.561283


In [21]:
# Example: find the top 10 similar song for 'A Day In The Life'
df = find_topk(song='A Day In The Life', similarity=similarity_df, k=10)
df

,Song,Similarity
0,Revolution,0.710186
1,Come Together,0.696557
2,Get Back,0.676128
3,Hello Goodbye,0.617065
4,Back In the U.S.S.R.,0.614700
5,Let It Be,0.601105
6,Hey Jude,0.599145
7,While My Guitar Gently Weeps,0.585429
8,Lucy In The Sky With Diamonds,0.577453
9,Here Comes The Sun,0.561283


In [22]:
backup = data

In [23]:
# Check if the similarity makes sense
q = """
    SELECT *,
        LEAD(song_played) OVER (PARTITION BY user_id ORDER BY time_played) AS next_song,
        LEAD(time_played) OVER (PARTITION BY user_id ORDER BY time_played) AS next_time
    FROM data
    """
data = pysqldf(q)

In [24]:
data[data.song_played == 'A Day In The Life'].groupby("next_song").id.count().sort_values(ascending = False).head()

next_song
Come Together    19
Revolution       15
Get Back         13
Let It Be        11
Hello Goodbye    10
Name: id, dtype: int64

## Q5: AB Testing Design
How would you set up a test to check whether your model works well and is improving
engagement?

**Assumptions:** each users are potentially independent, their journey aren't related to another users. Therefore, it is safe to take each user as our subject for the A/B Test
1. Evaluation metrics: engagement => average number of songs listened per user per day 
2. Population: all users in the US
3. Treament design: randomly split x% of the all the US users into 2 groups...One applying the algorithms, one applying the current default.
4. Other restrictions: a = 5%, power = 80% => Sample size...
5. after running some time, perform a one-tailed t-test on 'average #play per hour'\
    𝐻0 : population 'average #play per hour' is same in two groups\
    𝐻𝑎 : experiment group's population 'average #play per hour' is higher than control group's